In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import xgboost 
from neupy.estimators import mse
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dta = pd.read_excel('../Data/First_Seasonal.xlsx',index_col='Date')
# dta['Jan'] = dta['Month']==1
dta.head()

,Death,Death_rate,Incidence,Incidence_rate,forecast,Year,Month,error_ARIMA
Date,,,,,,,,
2005-01-01,143,0.011001,114194,8.784966,NaN,2005,1,NaN
2005-02-01,201,0.015463,87112,6.701542,NaN,2005,2,NaN
2005-03-01,281,0.021617,130134,10.011233,NaN,2005,3,NaN
2005-04-01,374,0.028772,133924,10.302798,NaN,2005,4,NaN
2005-05-01,388,0.029849,121326,9.333632,NaN,2005,5,NaN


In [4]:
# dataset = dta.loc['2006-02-1':'2014-12-01'][['Incidence_rate','forecast','Month']]
# x_train = dataset.loc[:'2012-12-01'][['forecast','Month']]
# y_train = dataset.loc[:'2012-12-01']['Incidence_rate']
# x_test = dataset.loc['2013-01-01':'2013-12-01'][['forecast','Month']]
# y_test = dataset['2013-01-01':'2013-12-01']['Incidence_rate']

In [19]:
dataset = dta.loc['2006-02-1':'2014-12-01'][['Incidence_rate','forecast','Month']]
x_train = dataset.loc[:'2012-12-01'][['forecast']]
y_train = dataset.loc[:'2012-12-01']['Incidence_rate']
x_test = dataset.loc['2013-01-01':'2013-12-01'][['forecast']]
y_test = dataset['2013-01-01':'2013-12-01']['Incidence_rate']

In [20]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear')
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [21]:
def try_std(n_estimators=20,max_depth=5,learning_rate=0.1):
    model = xgboost.XGBRegressor(objective='reg:linear',\
                                 n_estimators=n_estimators,\
                                 max_depth=max_depth,\
                                 learning_rate=learning_rate)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(mse(y_pred,y_test))

In [22]:
for n_estimators in range(20,40,10):
    for max_depth in range(5,20,5):
        for learning_rate in range(16,20,1):
            print(n_estimators,max_depth,learning_rate)
            try_std(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate/100)
            print('--\n')

20 5 16
0.130212357456641
--

20 5 17
0.1273668130327026
--

20 5 18
0.11599148203158662
--

20 5 19
0.11244363010350035
--

20 10 16
0.13047379750212004
--

20 10 17
0.1303825090297811
--

20 10 18
0.12543434706110768
--

20 10 19
0.11468161956934038
--

20 15 16
0.13047379750212004
--

20 15 17
0.1303825090297811
--

20 15 18
0.1254428166539099
--

20 15 19
0.11534375254462892
--

30 5 16
0.10773391868445871
--

30 5 17
0.1125257397078049
--

30 5 18
0.11339802947687355
--

30 5 19
0.11452243090822116
--

30 10 16
0.11663979222139614
--

30 10 17
0.12005896384358743
--

30 10 18
0.12012615862631043
--

30 10 19
0.11809585584307856
--

30 15 16
0.11734424015075695
--

30 15 17
0.12032150586208727
--

30 15 18
0.12272475578716589
--

30 15 19
0.12289355942312273
--



In [9]:
try_std(n_estimators=30,max_depth=5,learning_rate=0.16)

0.10932920540944902


In [23]:
# fit model no training data
model = xgboost.XGBRegressor(objective='reg:linear',n_estimators=30,max_depth=5,learning_rate=0.16)
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.16, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [25]:
dataset['XGBoost'] = model.predict(dataset[['forecast']])

In [26]:
mse(dataset['XGBoost'],dataset['Incidence_rate'])

0.0723838466846102

In [27]:
dataset.to_excel('../Data/First_Seasonal_XGBoost.xlsx')